In [ ]:
import nltk
from ebooklib import epub 
from  comFunc import *
import os 
import glob
from multiprocessing import Pool
import time
import re, string
import functools

In [ ]:
d_dir = '/home/some/txts/'
cwd = !pwd
cwd = cwd[0]
os.chdir(d_dir)
txtfiles = glob.glob('*.txt')
os.chdir(cwd)
print (len(txtfiles))
f = open('keywords.txt')
keywords = list(s.replace('\n','') for s in f.readlines())
f.close()

In [ ]:
def writeTag(fname,tag):
    key='http://purl.org/dc/elements/1.1/'
    try:
        book=epub.read_epub(fname)
        if 'subject' in book.metadata[key].keys():
            tgs=[b[0] for b in book.metadata[key]['subject']]
        else:
            tgs=[]
        ch_cnt=0
        for t in tag:
            if t not in tgs:
                if 'subject' in book.metadata[key].keys():
                    book.metadata[key]['subject'].append((t,{}))
                    ch_cnt+=1
                else:
                    book.metadata[key]['subject']=[(t,{})]
                    ch_cnt+=1
        if ch_cnt>0:
            tmp="/tmp/writeTag"+str(time.time())+'.epub'
            epub.write_epub(tmp,book)
            !mv "$tmp" "$fname"
    except :
        print('bad file  ',fname)
        #os.remove(fname)
def generate(f):
    tp = '/home/some/txt/'
    ep = '/home/some/epub/'
    ft = tp + f
    fe = ep + f[:-3] + "epub"
    #try:
    fl= open(ft,'r')
    txt=fl.read()
    fl.close()
    txt=''.join(''.join(tt for tt in t if tt in string.printable) for t in txt).lower()
    txt=txt.replace('\n',' ')
    txt=re.sub(r'[^a-zA-Z0-9 -]*','',txt)
    hist=nltk.FreqDist(txt.split())
    res=[[l,hist[l.strip()]] for l in keywords if l.strip() in hist.keys() ]
    tag=[r[0].strip() for r in res if r[1]>0]
    #tag=', '.join([r[0].strip() for r in res if r[1]>0])
    #print (txt)
    if len(tag)>0:
        #!ebook-meta "$fe" --tag "$tag" > /dev/null
        
        writeTag(fe,tag)
        return f[:-4],{r[0]:str(r[1]) for r in res},dict(hist)
    #except:
    #print('invalid   ',f)

In [ ]:
try:
    keys=load_obj('tagskeys')
except:
    keys=[]
len(keys)

In [ ]:
t0 = time.time()
p = Pool(10)
books=p.map(generate, list(filter(lambda x: x[:-4] not in keys, txtfiles))[0:1000])
tags={}
for book in books:
    try:
        if book[0] not in keys:
            tags[book[0]]=[book[1],book[2]]
    except:
        x=1
save_obj(tags, 'tags9')
keys=keys+list(tags.keys())
save_obj(keys,'tagskeys')
t1=time.time()
total = t1-t0
print('time: ',total)

In [ ]:
t0 = time.time()
g=generate(txtfiles[0])
print(type(g),len(g))
t1 = time.time()
total = t1-t0
print('time: ',total)

In [ ]:
len(list(filter(lambda x: x[:-4] not in bookTags.keys(), txtfiles))),len(txtfiles)

In [ ]:
def sliceDict(dct,st,en):
    keys=list(dct.keys())
    return {k:dct[k] for k in keys[st:en]}
d3=sliceDict(bookTags,4000,len(bookTags))
d4=sliceDict(bookTags,3000,4000)
d5=sliceDict(bookTags,2000,3000)
save_obj(d3,'tags3')
save_obj(d4,'tags4')
save_obj(d5,'tags5')

In [ ]:
save_obj(keys,'tagskeys')

In [ ]:
len(keys)

### loading ready tags

In [ ]:
books=load_obj('tags7')


In [ ]:
bookl=[]
for b in books:
    bookl.append([b,books[b][0]])
len(books)

In [ ]:
def wt(bb):
    #print(bb[0])
    tag=[t for t in bb[1].keys()]
    #print(bb[0],tag)
    if len(tag)>0:
        writeTag('/home/some/epub/'+bb[0]+".epub",tag)
p = Pool(10)

#map(functools.partial(generate, y=books[x]), a)
o=p.map(wt, bookl)

p.close()

In [ ]:
f=!find /home/some/epub/ -iname "Batt*"
f

In [ ]:
f

In [ ]:
key='http://purl.org/dc/elements/1.1/'
book=epub.read_epub(f[2])
if 'subject' in book.metadata[key].keys():
    tgs=[b[0] for b in book.metadata[key]['subject']]
else:
    tgs=[]


In [ ]:
    ch_cnt=0
    for t in tag:
        if t not in tgs:
            if 'subject' in book.metadata[key].keys():
                book.metadata[key]['subject'].append('t,{}')
                ch_cnt+=1
            else:
                book.metadata[key]['subject']=['t,{}']
                ch_cnt+=1
    if ch_cnt>0:
        tmp="/tmp/writeTag"+str(time.time())+'.epub'
        epub.write_epub(tmp,book)
        !mv "$tmp" "$fname"

    print('bad file  ',fname)    ch_cnt=0
    for t in tag:
        if t not in tgs:
            if 'subject' in book.metadata[key].keys():
                book.metadata[key]['subject'].append((t,{}))
                ch_cnt+=1
            else:
                book.metadata[key]['subject']=[(t,{})]
                ch_cnt+=1
    if ch_cnt>0:
        tmp="/tmp/writeTag"+str(time.time())+'.epub'
        epub.write_epub(tmp,book)
        !mv "$tmp" "$fname"

    print('bad file  ',fname)